# Introduction to Physics-informed neural networks

## 1- Introduction to neural networks

**Parametric statistics.** In supervised learning, the goal is to learn a function $f^\star$ linking the explanatory variable $X$ to the target variable $Y$, i.e., such that $Y = f^\star(X)+\epsilon$, where $\epsilon$ is a random noise.
In practice, $f^\star$ is estimated by $f_{\hat{\theta}}$, where $\hat{\theta}$ is a minimizer of the empirical risk
$$L(\theta) = \frac{1}{n}\sum_{i=1}^n |f_\theta(X_i)-Y_i|^2 + R(\theta),$$
where
* $(X_1, Y_1), \dots, (X_n,Y_n)$ are i.i.d. observations of $(X,Y)$,
* the function class $\{f_\theta,\; \theta\in \Theta\}$ is chosen to best approximate $f^\star$,
* $R(\theta)$ is a regularization term (e.g., Ridge, LASSO...) tailored to avoid overfitting.

**Neural networks.** Neural networks are a kind of parametric functions. They consist of a series of basic operations called *layers*, and the intermediate results of these layers are called the *neurons*. In what follows, we focus on *dense neural networks*, which are a succession of affine transformations and elementwise applications of the hyperbolic tangent (*tanh*) function.

**Question 1.** Let the function $f: \mathbb R^2 \to \mathbb R$ be defined by
$$f(t, x) = B_1 \tanh(A_1 t + A_2 x + A_3) + B_2 \tanh(A_4 t + A_5 x + A_6) + B_3 \tanh(A_7 t + A_8 x + A_9) + B_4.$$

a) Show that $f$ is a neural network.

b) What is the parameter $\theta$ such that $f = f_\theta$?

c) How many layers and neurons are there?



**Answer.**
<details>
  <summary>  (Click to show)</summary>
a) $f$ can be writen
$$f(t,x) = B \circ \tanh \circ A \begin{pmatrix} t\\ x\end{pmatrix}$$ where

*  $A \begin{pmatrix} t\\ x\end{pmatrix}= \begin{pmatrix} A_1 & A_2\\ A_4 & A_5 \\ A_7 & A_8\end{pmatrix} \begin{pmatrix} t\\ x\end{pmatrix} + \begin{pmatrix} A_3\\ A_6\\ A_9\end{pmatrix},$ is an affine transformation
* $\tanh (x_1, x_2, x_3) = (\tanh(x_1), \tanh(x_2), \tanh(x_3))$ is the elementwise application of the hyperbolic tangent,
* the neurons $z_1, z_2, z_3$ of the first layer are defined by $\begin{pmatrix} z_1\\ z_2\\z_3\end{pmatrix} =  \tanh \circ A \begin{pmatrix} t\\ x\end{pmatrix}$,
* and
$B \begin{pmatrix} z_1\\ z_2\\z_3\end{pmatrix}= \begin{pmatrix} B_1 & B_2 & B_3\end{pmatrix} \begin{pmatrix} z_1\\ z_2\\ z_3\end{pmatrix} + B_4$ is an affine transformation.

b) Here, the parameter $\theta$ are the parameters $(A_1, \dots, A_9, B_1, \dots, B_4)$ of the affine transformations.

c) Since the output space of $A: \mathbb R^2 \to \mathbb R^3$ is of dimension 3, there are 3 neurons in the first layers. Since we apply the activation function, tanh, only once, there is only one layer. Therefore, there is one layer with three neurons.
</details>

Neural networks are popular because they are a class of nonlinear functions, and computing their derivatives can be done efficiently using the so-called *backpropagation* algorithm. Most neural networks are implemented using the *Python* programming language. Currently, the two most popular deep learning libraries are *PyTorch* (torch) and *Tensorflow* (tf). In this tutorial, we will use PyTorch.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In what follows, we will rely heavily on the generation of random numbers. For example, when initializing a neural network, the weights are sampled according to a Gaussian distribution.
To ensure reproducibility of the results, we must keep track of the random numbers generated. We define a number, called a *seed*, which completely determines how the random number generator will be initialized.
Using the same seed will produce the same outputs.

In [ ]:
torch.manual_seed(314)

In [ ]:
# The next line means that we define a new type, called "NeuralNet", based on the
# structure of the neural network "nn.Module".
class NeuralNet(nn.Module):

# This new object will come with two functions "__init__" and "forward".
# "__init__" is useful to give an initial value to the variable "NeuralNet".

    def __init__(self):
        super(NeuralNet, self).__init__()

        # Here, we define the structure of the neural network.
        self.net = nn.Sequential(
            nn.Linear(2, 10),
            nn.Tanh(),
            nn.Linear(10, 10),
            nn.Tanh(),
            nn.Linear(10, 10),
            nn.Tanh(),
            nn.Linear(10, 1)
        )

    def forward(self, t, x):
        input = torch.cat([t, x], dim=1)
        return self.net(input)

**Question 2.** Detail the architecture of this neural network.

**Answer.**
<details>
  <summary>  (Click to show)</summary>
This neural network is a *dense* neural network with 3 layers, since we apply the activation function $\tanh$ three times. Each layer contains 10 neurons.
</details>

**GPU programming.** Graphics processing units (GPUs) are specialized electronic circuits designed to run some basic linear algebra operations more efficiently than cheaper, more common central processing units (CPUs). GPUs rely on both parallelization and efficient memory allocation. Pytorch provides a very simple way to run code on a GPU. The following command automatically detects whether a GPU is available, and sets the *device* variable to either the available GPU, or the CPU if no GPU is available.

In [ ]:
# Detect a NVIDIA GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

For MacOS users who want to use their M1, M2, or M3 GPU, the command is as follows.

In [ ]:
# Detect a GPU on MacOS computers
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

Then, the following line automatically samples a 2x2 array of uniform random variables on the *device*.

In [ ]:
test_variable = torch.rand(2, 2, device=device)
print(test_variable)

tensor([[0.7196, 0.6295],
        [0.6667, 0.3385]])


Note how lists of numbers are referred to as *tensors* in the PyTorch framework.
Tensors can represent many objects of linear algebra. For example,

* the vector $(1, 4)$ corresponds to $\hbox{tensor}([1, 4])$ of shape $(2,)$,
* the vector $\begin{pmatrix}1\\ 4\end{pmatrix}$  corresponds to $\hbox{tensor}([[1], [4]])$ of shape $(2,1)$,
* the matrix $\begin{pmatrix}1& 3\\ 4& -1\end{pmatrix}$ corresponds to $\hbox{tensor}([[1, 3], [4, -1]])$ of shape $(2,2)$.

We say that a tensor is of size $(*, 1)$ if its size is of the form $(n, 1)$.
Tensors supports various types of linear functions like

* addition and subtraction, encodes by the symbols + and -
* matrix-matrix and matrix-vector multiplications, encoded by the symbol @
* elementwise multiplication, encoded by the symbol *

## 2- PDE solving with PINNs

**PDE solving.** Here, we apply the tools from parametric statistics with neural networks to solve PDEs. To give a concrete example, let's solve the heat equation
$$\partial_1 f = \partial^2_{2,2} f$$
on the square $[0,1]^2$, with
* the boundary conditions $$\forall t\in [0,1],\; f(t, 0) = f(t, 1) = 0,$$
* and the initial conditions $$\forall x\in [0,1],\; f(0, x) = \sin(\pi x).$$

To solve this PDE, the *physics-informed neural networks* (PINNs) approach consists in minimizing the loss
$$L(\theta) = \frac{1}{n_i}\sum_{i=1}^{n_i}(f_\theta(0, X^{(ini)}_i)-\sin(\pi X^{(ini)}_i))^2 + \frac{1}{n_b}\sum_{j=1}^{n_b}f_\theta(T^{(b)}_j, 0)^2 + \frac{1}{n_c}\sum_{k=1}^{n_c}(\partial_t f_\theta(T^{(c)}_k, X^{(c)}_k)-\partial^2_{x,x} f_\theta(T^{(c)}_k, X^{(c)}_k))^2,$$
where the points $X^{(ini)}_i$, $T^{(b)}_j$, $T^{(c)}_k$, and $X^{(c)}_k$ are independent and sampled along the uniform distribution on $[0,1]$.

**Question 3.** Given a fixed parameter $\theta$, what is the limit of $L(\theta)$ as $n_i, n_b, n_c \to \infty$?

**Answer.**
<details>
  <summary>  (Click to show)</summary>
The law of large number states that the empirical risk $L(\theta)$ converges almost surely to the theoretical risk $R(\theta)$ defined by
$$\lim_{n_b, n_c \to \infty} L(\theta) = R(\theta) = \int_0^1(f_\theta(0, x)-\sin(\pi x))^2dx + \int_0^1 f_\theta(t, 0)^2dt + \int_0^1(\partial_t f_\theta(t,x)-\partial^2_{x,x} f_\theta(t,x))^2dtdx.$$
</details>

**Question 4.** Let $R(\theta) = \lim_{n_i, n_b, n_c\to \infty} L(\theta)$. Why does minimizing $R$ amounts to solving the PDE?

**Answer.**
<details>
  <summary>  (Click to show)</summary>
Note that $\forall \theta, \;R(\theta) \geq 0$. Morevoer, $R(\theta) = 0$ if and only if $f_\theta$ satifies the initial conditions, the boundary conditions, and the PDE.
</details>

**Question 5.** Implement a function *pde_residual* computing the PDE residual $\partial_t u - \partial^2_{x,x}u$, where $u$ is a neural network. To do so, use the following framework of Pytorch, which allows to compute the derivarive of $u$ with respect to a variable $t$.

In [ ]:
# PDE residual
def pde_residual(model, t, x):
    # t and x are tensors of size (*, 1)
    u = model(t, x) # Evaluate the neural network u at the values where you want to compute the derivatives
    t.requires_grad_(True) # Allows to compute the derivative with respect to the variable t

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    # Compute the gradient of u with respect to t.
    # Grad_outputs specify the shape of the gradient.
    # retain_graph allows to differentiate multiple times.

    u_x = ...
    u_xx = ...

    return u_t - u_xx

**Question 6.** Implement a function that generates the training data using the following functions from the *torch* library: *rand*, *randn*, *zeros*, *ones*, *linspace*, and *sin*. As is typical with neural networks, all tensors must be of shape $(*, 1)$.

In [ ]:
# Training data
def generate_training_data(N_f=10**4, N_b=10**2, N_i=10**2):
    # Collocation points (interior)
    t_f = torch.rand(N_f, 1, device=device)
    x_f = ...

    # Boundary conditions
    t_b = ...
    x_b0 = ...
    x_b1 = ...

    # Initial condition
    x_i = ...
    t_i = ...
    u_i = ...

    return t_f, x_f, t_b, x_b0, x_b1, t_i, x_i, u_i

**Question 7.** Implement the gradient descent to train the neural network.

In [ ]:
# Training loop
def train(model, epochs=5000, lr=1e-3):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    t_f, x_f, t_b, x_b0, x_b1, t_i, x_i, u_i = generate_training_data(N_f=10**4, N_b=10**2, N_i=10**2)

    for epoch in range(epochs):

        optimizer.zero_grad()

        # PDE residual loss
        loss_f = ...

        # Boundary condition loss
        loss_b = ...

        # Initial condition loss
        loss_i = ...

        loss = loss_f + loss_b + loss_i
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4e}")

    print("Training complete.")

**Question 8.** Reset the seed to 314. Then, train your neural network.

**Analytical solutions.**
To estimate the performance of the PINN, we can compare its outputs with the unique solution $f^\star$ of the PDE $$f^\star(t,x) = \exp(-\pi^2 t)\sin(\pi x).$$

**Question 9.** Check that $f^\star$ is indeed a solution of the PDE.

**Answer.**
<details>
  <summary>  (Click to show)</summary>
On the one hand,
$$\partial_t f = -\pi^2 f.$$
On the other hanf,
$$\partial^2_{x,x} f = -\pi^2 f.$$
Thus, $\partial_t f = \partial^2_{x,x} f$.
</details>

**Question 10.** Using three plots, plot the functions $f^\star$, the PINN, and the error $|\hbox{PINN}-f^\star|$. Compute the Mean Average Error (MAE) $$MAE = \frac{1}{100^2}\sum_{k=0}^{100} \sum_{\ell=0}^{100} |\hbox{PINN}(k/100, \ell/100)-f^\star(k/100, \ell/100)|.$$

In [ ]:
def plot(model):
  ...

In [ ]:
plot(model)

# 3- Some basic improvements over vanilla PINNs

There are many ways to improve this implementation, but which complicates the understanding of the code baseline.
For instance, instead of fixing the collocation points, and the points penalizing the boundary and initial conditions, we can sample new points at each epoch. This has the advantage of both increasing the number of available data while reducing overfitting.

**Question 11.** Implement this PINN where data are resampled at each epoch.

In [ ]:
def train_2(model, epochs=5000, lr=1e-3):



**Question 12.** Train your model and compare its MAE with the other PINN.

## 4- Hybrid modelling

**Hybrid modelling.** Here, we assume not to know the boundary conditions entirely. Instead, we have at hand $n$ i.i.d. observations $(T_1, X_1, Y_1)$, $\dots$, $(T_n, X_n, Y_n)$ such that

* $(T, X$) follows the uniform distribution on $[0,1]^2$
* $Y = f^\star(T, X)+\epsilon$,  
* and $f^\star$ is a solution to the heat equation $$\partial_1 f = \partial^2_{2,2} f.$$

In the next example, the unknown function $f^\star$ is once again
$$f^\star(t,x) = \exp(-\pi^2t)\sin(\pi x)$$
and $\epsilon$ is a gaussian noise $N(0,1)$.

**Question 13.** Which empirical loss corresponds to this problem?

**Answer.**
<details>
  <summary>  (Click to show)</summary>
$$L(\theta) = \frac{1}{n}\sum_{i=1}^{n}(f_\theta(T_i, X_i)-Y_i))^2 + \frac{\lambda_n}{n_c}\sum_{k=1}^{n_c}(\partial_t f_\theta(T^{(c)}_k, X^{(c)}_k)-\partial^2_{x,x} f_\theta(T^{(c)}_k, X^{(c)}_k))^2,$$
where the points $T^{(c)}_k$ and $X^{(c)}_k$ are independent and sampled along the uniform distribution on $[0,1]$. $\lambda_n \geq 0$ is an hyperparameter weighting the importance of the PDE penalty.
</details>

**Question 14.** Implement a function to generate the training data using the functions *rand*, *randn*, *zeros*, *ones*, *linspace* and *sin* of the *torch* library. As usual with neural networks, all tensors must be of shape $(*, 1)$.

In [ ]:
# Training data
def generate_training_data_2(N_f=10**4, N_data=10**2, sigma=0.1):
    # Collocation points (interior)
    t_f = ...
    x_f = ...

    # Data points
    t = ...
    x = ...
    y = ...

    return t_f, x_f, t, x, y


**Question 15.** To assess the impact of using the physical a priori on the learning performance, implement two algorithms: one which uses the physical penalty (i.e., $\lambda_n > 0$) and one which does not (i.e., $\lambda_n = 0$).

In [ ]:
def train_without_PDE(model, epochs=5000, lr=1e-3):

def train_hybrid(model, epochs=5000, lr=1e-3):


**Question 16.** Compare their performance.

# 5- Going Further

PINNs are interesting because they adapt seamlessly to many contexts in PDE solving and hybrid modelling without  increasing the complexity of the algorithm.


**Question 17: Nonlinear PDEs** Nonlinear PDEs are notoriously difficult to solve. However, implementing PINNs to solve nonlinear PDEs is straightforward.  Implement a PINN to solve the Burgers equation $$\partial_1 f + f \partial_2 f = 0$$ on the square $[0,1]$ with initial conditions $f(0,x) = 1+x$ and boundary conditions $$f(t,0) = \frac{1}{1+t}.$$ Compare it with the values of the unique solution $f(t,x) = \frac{1+x}{1+t}$.
(Hint: You only need to change the function *pde_residual* and to get rid of the boundary and initial conditions).

**Question 18: General geometries.** Solving PDEs on complex geometries often results in complicated meshes when using classical techniques However, PINNs can be easily implemented to solve PDEs on general geometries. Implement a PINN to solve the Laplace equation $$\partial^2_{1,1} f + \partial^2_{2,2} f + \partial^3_{3,3} f = 6$$ on the ball $\{(x_1, x_2, x_3)\in \mathbb R^3, \; x_1^2 + x_2^2+x_3^3 \leq 1\}$ with boundary conditions
* $f(x_1,x_2,x_3) = x_1^2+x_2^2+x_3^2$ if $x_1^2+x_2^2+x_3^2 = 1$.

Compare these values with those of the solution $f(x_1,x_2, x_3) = x_1^2+x_2^2+x_3^2$.